# Sonraki Kelime Tahmin Modeli
Bu model normalde bir metin yazma modeli olarak kullanılmaktadır. 
Ancak bu projede sonraki kelimeyi değil önceki kelimeyi tahmin ederek şiir yazmaya çalışıyoruz. 
Çünkü şiirde satır sonlarındaki kelimelerin kafiyeli olması beklenir. 
Dolayısıyla bir kelime seçip o kelimeyi satır sonuna koyarak önceki kelimeleri tahmin edip bir şiir dizesi oluşturuyoruz. 
Bir sonraki satırda da yine kafiyeli bir kelimeden yola çıkarak öndeki kelimer sırayla tahmin edilir ve şiir tamamlanır.
<br><br>
Bu modeli eğitmek için Yunus Emre şiirleri kullanılmıştır. Şiirler başlıksız olarak alt alta eklenip bir metin dosyasına yerleştirildi.
### Gerekli kütüphaneleri ekleyip şiirleri okuyarak işe koyulalım

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Read the text file
with open('yunus.txt', 'r', encoding='utf-8') as file:
    text = file.read()

In [2]:
# Metinde gözden kaçan rakamlar varsa onları temizleyelim
import re
text=re.sub(r'\d+', '', text)

In [3]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1
total_words

4604

In [4]:
# Burada kitabın her bir satırını alıp kelimelerden ngram dizileri oluşturuyor. 
# Yani satırdaki kelimeleri sayıyay dönüştürüp (Örn:1, 1561, 5, 129, 34) bu sayılardan 2, 3, 4 ve 5 gram lı diziler oluşturuyor.
#[1, 1561],
#[1, 1561, 5],
#[1, 1561, 5, 129],
#[1, 1561, 5, 129, 34]

# Sonraki kelimeyi tahmin etmek için aşağıdaki kod kullanılmalıdır.
#input_sequences = [tokenizer.texts_to_sequences([line])[0][:i+1] for line in text.split('\n') 
#                   for i in range(1, len(tokenizer.texts_to_sequences([line])[0]))]

# Ancak biz önceki kelimeyi tahmin etmek istediğimiz için sıralamayı ters çeviriyoruz.
input_sequences = [tokenizer.texts_to_sequences([line])[0][i::-1] for line in text.split('\n') 
                   for i in range(1, len(tokenizer.texts_to_sequences([line])[0]))]

In [5]:
# Yukarıda oluşturduğumuz input_sequences dizisi farklı boyutlardan oluşan dizi elemanlarına sahip.
# Bunlar içerisinde en uzun dizinin kaç elemanı olduğunu bulalım. 
max_sequence_len = max([len(seq) for seq in input_sequences])
max_sequence_len

16

In [6]:
# Yukarıda oluşturduğumuz input_sequences dizisinin eleman sayılarını eşitliyoruz. 
# Bunu yaparken baş kısımlara (padding='pre') 0 değerleri atanır.
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
input_sequences

array([[   0,    0,    0, ...,    0,  818, 1424],
       [   0,    0,    0, ...,  819,  818, 1424],
       [   0,    0,    0, ...,  819,  818, 1424],
       ...,
       [   0,    0,    0, ...,    0,   80, 1422],
       [   0,    0,    0, ..., 1423,   80, 1422],
       [   0,    0,    0, ..., 1423,   80, 1422]])

In [7]:
# Oluşturacağımız modelde girdi değerlerimiz oluşturduğumuz input_sequences dizinin her bir satırının ilk 15 elemanı
# Çıktı değişkenimiz ise bu dizinin 16. elemanları
X = input_sequences[:, :-1]
y = input_sequences[:, -1]

In [8]:
# Burada hedef değişkeni one-hot kodlamasına dönüştürüyoruz. 
# Bu, her hedef değeri için bir vektör oluşturur ve hedef değerinin indeksine karşılık gelen yerde 1 ve diğer tüm yerlerde 0 içerir.
y = np.array(tf.keras.utils.to_categorical(y, num_classes=total_words))
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

### Modelleme

In [9]:
model = Sequential()
model.add(Embedding(total_words, 100, input_shape=(max_sequence_len-1,)))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))

C:\ProgramData\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:81: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [10]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=100, verbose=1)

Epoch 1/100
304/304 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.0394 - loss: 7.0914
Epoch 2/100
304/304 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.0485 - loss: 5.7974
Epoch 3/100
304/304 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.0568 - loss: 5.5084
Epoch 4/100
304/304 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.0742 - loss: 5.1918
Epoch 5/100
304/304 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.1079 - loss: 4.9103
Epoch 6/100
304/304 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.1475 - loss: 4.5993
Epoch 7/100
304/304 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.1917 - loss: 4.2773
Epoch 8/100
304/304 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.2630 - loss: 3.8985
Epoch 9/100
304/304 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.3461 - loss: 3.4892
Epoch 10/100
304/304 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.4235 - loss: 3.0769
Epoch 11/100
304/304 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.5041 - loss: 2.6870
Epoch 12/100
304/304 ━━━━━━━━━━━━━━━━━━━━

### Sonuç:
Geliştirilen modelde Yunus Emre şiirlerini taklit eden ve satır sonunda başlayarak bir önceki kelimeyi tahmin eden bir araç geliştirilmiştir. 
<br><br>Modeli kaydedip streamlit uygulamasına dönüştürelim. Uygulamada kullanılmak üzere tokenizer nesnesi ve bu nesnede kullanılmak üzere kelime-index sözlüğü kaydedilmelidir. Tek başına model yeterli değildir.

In [12]:
# Modelin kaydedilmesi
model.save('yunus_model.h5')

In [13]:
#Tokenizer nesnesinin kaydedilmesi
import pickle
pickle.dump(tokenizer, open("yunus_tokenizer.pkl", "wb"))

In [14]:
# Kelime-index sözlüğünün kaydedilmesi
import pandas as pd
word_index_df = pd.DataFrame(list(tokenizer.word_index.items()), columns=['word', 'index'])
# Sözlükte rakam istemiyorsanız aşağıdaki satırı kullanabilirsiniz.
word_index_df = word_index_df[~word_index_df["word"].str.isnumeric()]
word_index_df.to_csv('yunus_word_index.csv', index=False)
word_index_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4603 entries, 0 to 4602
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   word    4603 non-null   object
 1   index   4603 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 72.1+ KB


### Test Edelim

In [15]:
import random

word_index_df = word_index_df[~word_index_df["word"].str.isnumeric()]

def kafiye(kelime):
    k=random.randint(1, 2)
    return word_index_df[word_index_df["word"].str.endswith(kelime[-k:])].sample().iloc[0]["word"]

def pre_words(seed_text, n):
    for _ in range(n):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = np.argmax(model.predict(token_list), axis=-1)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

In [ ]:
i=1
onceki=""
next="dünya"
siir=[]
while(i<13):
    dize=pre_words(next, 5)
    next=kafiye(dize.split()[0])
    if(onceki!=dize):
        siir.append((" ".join(dize.split()[::-1])).capitalize())
        onceki=dize
        i=i+1
    if((i-1)%4==0):
        siir.appen("")